In [19]:
import pandas as pd

In [20]:
cleaned_league = pd.read_csv("complete_cleaned_to_25.csv")
counter_df = pd.read_csv("champs_counters_wr.csv")
cleaned_league.head()

,Unnamed: 0,teamname,league,year,gameid,side,champion,ban1,ban2,ban3,...,opp_csat25,golddiffat25,xpdiffat25,csdiffat25,killsat25,assistsat25,deathsat25,opp_killsat25,opp_assistsat25,opp_deathsat25
0,0,I May,LPL,2017,1506-1540,Blue,Maokai,Syndra,Malzahar,Ashe,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Royal Never Give Up,LPL,2017,1506-1540,Red,Kled,Camille,Rengar,Zyra,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,I May,LPL,2017,1506-1541,Blue,Maokai,Syndra,Malzahar,Ashe,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Royal Never Give Up,LPL,2017,1506-1541,Red,Trundle,Rengar,Camille,Varus,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Invictus Gaming,LPL,2017,1507-1544,Blue,Singed,Jayce,Elise,Malzahar,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
cleaned_league['year'].unique()

array([2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025])

In [22]:
cleaned_league = cleaned_league.drop(columns=["Unnamed: 0"])

In [24]:
import pandas as pd
import ast  

def format_champion(champ):
    if pd.isna(champ):  
        return None  
    return champ.lower().replace(" ", "-").replace("'", "").replace("nunu-&-willump", "nunu-willump").replace("dr.-mundo", "dr-mundo")

def get_picks_and_bans(df): 
    blue_picks = df[df['side'] == 'Blue'][['pick1', 'pick2', 'pick3', 'pick4', 'pick5']].values.flatten().tolist()
    red_picks = df[df['side'] == 'Red'][['pick1', 'pick2', 'pick3', 'pick4', 'pick5']].values.flatten().tolist()

    blue_bans = df[df['side'] == 'Blue'][['ban1', 'ban2', 'ban3', 'ban4', 'ban5']].values.flatten().tolist()
    red_bans = df[df['side'] == 'Red'][['ban1', 'ban2', 'ban3', 'ban4', 'ban5']].values.flatten().tolist()

    blue_picks = [format_champion(champ) for champ in blue_picks if pd.notna(champ)]
    red_picks = [format_champion(champ) for champ in red_picks if pd.notna(champ)]
    
    blue_bans = [format_champion(champ) for champ in blue_bans if pd.notna(champ)]
    red_bans = [format_champion(champ) for champ in red_bans if pd.notna(champ)]

    return blue_picks, red_picks, blue_bans, red_bans

def preprocess_counters(counter_df):
    counter_dict = {}
    wr_dict = {}  # Store win rates for each champion

    for _, row in counter_df.iterrows():
        champ = format_champion(row['Champion'])
        wr_dict[champ] = row['champ_wr']  # Store win rate
        
        if pd.notna(row['counters champ']) and isinstance(row['counters champ'], str):
            try:
                counters = ast.literal_eval(row['counters champ'])  
                if isinstance(counters, tuple):  # Ensure it's a tuple before extracting
                    if champ not in counter_dict:
                        counter_dict[champ] = set()
                    counter_dict[champ].add(format_champion(counters[0]))  # Extract champion names
            except (SyntaxError, ValueError):
                continue  # Skip malformed rows
                
    return counter_dict, wr_dict  # Return both counter and win rate dictionaries

def count_counters(df, counter_dict): 
    blue_picks, red_picks, blue_bans, red_bans = get_picks_and_bans(df)

    num_blue_counters = sum(1 for red_champ in red_picks for blue_champ in blue_picks if blue_champ in counter_dict.get(red_champ, set()))
    num_red_counters = sum(1 for blue_champ in blue_picks for red_champ in red_picks if red_champ in counter_dict.get(blue_champ, set()))
    
    num_blue_counters_banned = sum(1 for blue_champ in blue_picks for ban in blue_bans if ban in counter_dict.get(blue_champ, set()))
    num_red_counters_banned = sum(1 for red_champ in red_picks for ban in red_bans if ban in counter_dict.get(red_champ, set()))

    return pd.Series({
        "Red Counters": num_red_counters, 
        "Blue Counters": num_blue_counters,
        "Red Counters Banned": num_red_counters_banned,
        "Blue Counters Banned": num_blue_counters_banned
    })

def calculate_mean_wr(df, wr_dict):
    blue_picks, red_picks, _, _ = get_picks_and_bans(df)

    # Get WR values for each pick, default to 0.5 if missing
    blue_wr = [wr_dict.get(champ, 0.5) for champ in blue_picks]
    red_wr = [wr_dict.get(champ, 0.5) for champ in red_picks]

    mean_blue_wr = sum(blue_wr) / len(blue_wr) if blue_wr else 0
    mean_red_wr = sum(red_wr) / len(red_wr) if red_wr else 0

    return pd.Series({"Mean Blue WR": mean_blue_wr, "Mean Red WR": mean_red_wr})

def process_data(df, counter_df):
    counter_dict, wr_dict = preprocess_counters(counter_df)
    
    # Compute counters and merge
    counters_df = df.groupby("gameid").apply(lambda x: count_counters(x, counter_dict)).reset_index()
    df = df.merge(counters_df, on="gameid", how="left")
    
    df["num_counters_picked"] = df.apply(lambda row: row["Red Counters"] if row["side"] == "Red" else row["Blue Counters"], axis=1)
    df["num_counters_banned"] = df.apply(lambda row: row["Red Counters Banned"] if row["side"] == "Red" else row["Blue Counters Banned"], axis=1)
    
    df["num_counters_picked"] = df["num_counters_picked"].fillna(0)
    df["num_counters_banned"] = df["num_counters_banned"].fillna(0)
    
    # Calculate PGA
    df["PGA"] = df["num_counters_picked"] + df["num_counters_banned"]
    
    df.drop(columns=["Red Counters", "Blue Counters", "Red Counters Banned", "Blue Counters Banned"], inplace=True)

    # Compute mean WR for each game and merge
    wr_df = df.groupby("gameid").apply(lambda x: calculate_mean_wr(x, wr_dict)).reset_index()
    df = df.merge(wr_df, on="gameid", how="left")

    # Assign WR values based on team side
    df["mean_champ_wr"] = df.apply(lambda row: row["Mean Red WR"] if row["side"] == "Red" else row["Mean Blue WR"], axis=1)

    df.drop(columns=["Mean Red WR", "Mean Blue WR"], inplace=True)

    # Compute which team had the higher PGA
    df['higher_PGA'] = df.groupby('gameid')['PGA'].transform(lambda x: (x == x.max()).astype(int))

    # Remove games where both teams have the same PGA
    same_pga_games = df.groupby('gameid')['PGA'].nunique() == 1
    df = df[~df['gameid'].isin(same_pga_games[same_pga_games].index)]
    
    return df  

final = process_data(cleaned_league, counter_df)
final.to_csv('transformed_data_to_25.csv')
final

C:\Users\Viki Shi\AppData\Local\Temp\ipykernel_16052\363230576.py:76: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  counters_df = df.groupby("gameid").apply(lambda x: count_counters(x, counter_dict)).reset_index()
C:\Users\Viki Shi\AppData\Local\Temp\ipykernel_16052\363230576.py:91: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  wr_df = df.groupby("gameid").apply(lambda x: calculate_mean_wr(x, wr_dict)).rese

,teamname,league,year,gameid,side,champion,ban1,ban2,ban3,ban4,...,assistsat25,deathsat25,opp_killsat25,opp_assistsat25,opp_deathsat25,num_counters_picked,num_counters_banned,PGA,mean_champ_wr,higher_PGA
0,I May,LPL,2017,1506-1540,Blue,Maokai,Syndra,Malzahar,Ashe,Karma,...,NaN,NaN,NaN,NaN,NaN,0,2,2,0.508666,1
1,Royal Never Give Up,LPL,2017,1506-1540,Red,Kled,Camille,Rengar,Zyra,Elise,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0.493086,0
2,I May,LPL,2017,1506-1541,Blue,Maokai,Syndra,Malzahar,Ashe,Rek'Sai,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0.510833,0
3,Royal Never Give Up,LPL,2017,1506-1541,Red,Trundle,Rengar,Camille,Varus,Cassiopeia,...,NaN,NaN,NaN,NaN,NaN,1,0,1,0.492558,1
4,Invictus Gaming,LPL,2017,1507-1544,Blue,Singed,Jayce,Elise,Malzahar,Kha'Zix,...,NaN,NaN,NaN,NaN,NaN,0,1,1,0.492380,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120123,Vitality.Bee,LFL,2025,LOLTMNT06_96854,Red,K'Sante,Skarner,Viktor,Maokai,Rell,...,0.0,1.0,0.0,2.0,0.0,1,1,2,0.497897,1
120124,Zerance,LFL2,2025,LOLTMNT06_96867,Blue,Rumble,Aurora,Karthus,Kayn,Galio,...,4.0,3.0,0.0,5.0,1.0,1,2,3,0.501714,1
120125,Project Conquerors,LFL2,2025,LOLTMNT06_96867,Red,Jax,Viktor,K'Sante,Ivern,Jayce,...,5.0,1.0,5.0,4.0,3.0,2,0,2,0.509128,0
120128,IZI Dream,LFL2,2025,LOLTMNT06_96906,Blue,Rumble,Akali,Rell,Varus,Skarner,...,1.0,2.0,1.0,1.0,0.0,1,2,3,0.516646,1


In [18]:
final.isna().sum()

teamname               22
league                  0
year                    0
gameid                  0
side                    0
                       ..
num_counters_picked     0
num_counters_banned     0
PGA                     0
mean_champ_wr           0
higher_PGA              0
Length: 67, dtype: int64